In [7]:
import numpy as np
import pandas as pd
import os
import math
from time import time
import datetime as dt
#定义几个函数
# def find_key(keys,obj):  二分查找user id为obj的用户是否在训练集用户列表keys中出现
# def softmax(vec): 9类别的时长转softmax概率，时长太大，指数溢出暂时不用
# def cnt2pro(vec): 9类别的时长转概率，分别除以9类别时长总和
# def close_txt(id_cnt_files): 一次性关闭多个txt文件，id_cnt_files为文件列表
# def getfiles(mode): 返回文件名列表，mode："train", "val", "test"

def find_key(keys,obj):
    len_keys=len(keys)
    l=0
    r=len_keys-1
    while True:
        if r-l<3:
            break
        mid=(l+r)//2
        if keys[mid]==obj:
            return True
        if keys[mid]<obj:
            l=mid+1
        else:
            r=mid+1
    for k in range(l,r+1):
        if keys[k]==obj:
            return True
    return False    
def softmax(vec):
    return_ans=np.zeros([vec.shape[0]])
    for k in range(vec.shape[0]):
        SUM=0.0
        for n in range(vec.shape[0]):
            SUM+=math.exp(vec[n]-vec[k])
        return_ans=1.0/SUM
    return return_ans
def cnt2pro(vec):
    return_ans=np.zeros([vec.shape[0]])
    SUM=0.0
    for n in range(vec.shape[0]):
        SUM+=vec[n]
    for k in range(vec.shape[0]):
        return_ans[k]=vec[k]/SUM
    return return_ans
def close_txt(id_cnt_files):
    for file in id_cnt_files:
        file.close()
def getfiles(mode):
    def get_name_from_txt(file_name):
        name_id=[]
        f=open(file_name)
        lines=f.readlines()
        for line in lines:
            s=line.split(".jpg")[0]
            s=s.split("/")[-1]
            name_id.append(s)
        f.close()
        return name_id
    return_ans=[]
    if mode=="test":
        for i in range(2):
            path="/root/userfolder/luotao/final/test_visit_"+str(i)+"/"+str(i)+"/"
            files=os.listdir(path)
            for file in files:
                return_ans.append(path+file)
        return_ans.sort()
        return return_ans
    elif mode=="train":
        file_name="/root/userfolder/luotao/final_code/data_txt/train.txt"
    elif mode=="val":
        file_name="/root/userfolder/luotao/final_code/data_txt/val.txt"
    name_id=get_name_from_txt(file_name)
    name2file={}
    for i in range(10):
        path="/root/userfolder/luotao/final/train_visit_"+str(i)+"/"+str(i)+"/"
        files=os.listdir(path)
        for file in files:
            s=file.split(".txt")[0]
            name2file[s]=path+file
    return_ans=[]
    for name in name_id:
        return_ans.append(name2file[name])
    return return_ans

holiday = ['20181229','20181230','20181231','20190101','20180930','20181001','20181002','20181003','20181004','20181005','20181006',
           '20181007','20190203','20190204','20190205','20190206','20190207','20190208','20190209','20190210']

In [8]:
# 对训练集进行处理的代码
# 遍历训练集的原始visit txt文件，分9个类别记录每个用户的总时长
# 输出9个txt，每个文件对应一个类别，每个文件中两列记录用户id和时长
# 分两步进行：第一步：遍历txt，第二步：对于每个类别，依据时长对用户排序，删除时长低的80% 或 90% 的用户，再合并同一类别中的相同用户，时长相加

###################################################第一步

id_cnt_files=[]
for i in range(1,10):
    id_cnt_file=open("id_cnt/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
files=getfiles("train")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()

for file in files:
    label=int(file.split(".txt")[0].split("/")[-1].split("_")[1])
    table = pd.read_table(file, header=None)
    tv=table.values
    user_num=tv.shape[0]
    temp=""
    
    for i in range(user_num):
        L = tv[i,1].split(',')
        day = 0
        for item in L: 
            d = dt.datetime.strptime(item.split('&')[0],"%Y%m%d")
            if d.weekday() in [5,6]: day+=1
        if day==0: continue
        temp+=str(tv[i,0])+","+str(day)+"\n"#str(tv[i,1].count("&"))+tv[i,1].count("|")) +"\n" # 规则核心
    id_cnt_files[label-1].write(temp)
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1
close_txt(id_cnt_files)

350000
0.0000% 0.04976534843444824
0.2857% 152.2736337184906
0.5714% 158.12258219718933
0.8571% 161.7949616909027
1.1429% 154.67799854278564
1.4286% 143.26864409446716
1.7143% 146.11918568611145
2.0000% 150.64921855926514
2.2857% 150.16542506217957
2.5714% 154.9714663028717
2.8571% 150.82585287094116
3.1429% 156.77001452445984
3.4286% 155.13944935798645
3.7143% 143.8172206878662
4.0000% 147.7700912952423
4.2857% 153.66875648498535
4.5714% 146.18547916412354
4.8571% 154.17263507843018
5.1429% 147.88424277305603
5.4286% 156.84671831130981
5.7143% 159.9155149459839
6.0000% 156.70820546150208
6.2857% 159.64154362678528
6.5714% 145.68429970741272
6.8571% 170.75153255462646
7.1429% 210.90320420265198
7.4286% 183.73846912384033
7.7143% 191.33717679977417
8.0000% 203.0335817337036
8.2857% 205.26740288734436
8.5714% 223.74406147003174
8.8571% 152.68896460533142
9.1429% 140.43077659606934
9.4286% 135.63031792640686
9.7143% 136.70781660079956
10.0000% 144.89019012451172
10.2857% 152.8115127086639

85.1429% 192.79144835472107
85.4286% 200.57210636138916
85.7143% 210.11979937553406
86.0000% 246.51371121406555
86.2857% 236.10683059692383
86.5714% 210.99492645263672
86.8571% 217.8402464389801
87.1429% 204.98655652999878
87.4286% 207.20755648612976
87.7143% 206.30952763557434
88.0000% 190.63013172149658
88.2857% 207.2473545074463
88.5714% 214.79496908187866
88.8571% 196.06616282463074
89.1429% 142.76468443870544
89.4286% 137.3215925693512
89.7143% 133.44710445404053
90.0000% 139.66874527931213
90.2857% 127.27157330513
90.5714% 140.7901327610016
90.8571% 156.4570667743683
91.1429% 137.978093624115
91.4286% 149.2771120071411
91.7143% 136.5440797805786
92.0000% 128.00707173347473
92.2857% 146.25315237045288
92.5714% 131.289160490036
92.8571% 134.57393407821655
93.1429% 132.2097408771515
93.4286% 133.93899655342102
93.7143% 137.70780062675476
94.0000% 127.40647053718567
94.2857% 137.9520139694214
94.5714% 166.10356426239014
94.8571% 166.21070981025696
95.1429% 180.03412175178528
95.4286%

In [9]:
###################################################第二步
last_time=time()
for i in range(1,10):
    print("id_cnt/id_cnt_"+str(i)+".txt")
    table = pd.read_csv("id_cnt/id_cnt_"+str(i)+".txt",sep=",", header=None)
    table.columns = ['uid','num']
    df = table.groupby(['uid'])['num'].sum().reset_index(name='cnt_num').sort_values(by='cnt_num',ascending=True)
    print("去重前:",len(table))
    print("去重后:",len(df))
    # 写入del_same
    df.to_csv("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None,index=None)
    now_time=time()
    print("%d"%(i),now_time-last_time)

print("time: ",time()-last_time)


id_cnt/id_cnt_1.txt
去重前: 71824290
去重后: 40443199
1 730.3939259052277
id_cnt/id_cnt_2.txt
去重前: 55834509
去重后: 21778332
2 1153.835517168045
id_cnt/id_cnt_3.txt
去重前: 13383509
去重后: 6212679
3 1232.9879426956177
id_cnt/id_cnt_4.txt
去重前: 6582682
去重后: 4098119
4 1276.1149694919586
id_cnt/id_cnt_5.txt
去重前: 1733369
去重后: 664040
5 1283.5911169052124
id_cnt/id_cnt_6.txt
去重前: 14763207
去重后: 9202948
6 1387.278389453888
id_cnt/id_cnt_7.txt
去重前: 35692278
去重后: 17505829
7 1646.5878937244415
id_cnt/id_cnt_8.txt
去重前: 5630444
去重后: 4030423
8 1687.707465171814
id_cnt/id_cnt_9.txt
去重前: 26222124
去重后: 13829499
9 1862.0135910511017
time:  1862.0144641399384


In [10]:
# 测试代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
T = 0
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_csv("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    table.columns = ['user_id','num']
    #T = table['num'].mean()
    print(len(table))
    print("阈值为 %d"%T)
    print(len(table[table.num >T]))
    table = table[table.num >T]
    temp=table.values[:,:]
    user_num=temp.shape[0]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        user2hours[i-1][temp[j,0]]=int(temp[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

id2label=np.zeros([100000])
id2pro=np.zeros([100000,9])
id2num=np.zeros([100000,9])
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("test")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    reg_id=int(file.split(".txt")[0].split("/")[-1])
    table = pd.read_table(file, header=None)
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        L = table2cnt[i,1].split(',')
        day = 0
        for item in L: 
            d = dt.datetime.strptime(item.split('&')[0],"%Y%m%d")
            if d.weekday() in [5,6]: day+=1
        table2cnt[i,1]=day
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0   #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user): #每个用户有9个时长
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label #reg_per_label 在 9 个类别所有用户总时长
    id2num[reg_id]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[reg_id]=pre_label
    id2pro[reg_id]=pro[:]
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1

np.save("id2num_test.npy",id2num)
np.save("id2label_test.npy",id2label)
np.save("id2pro_test.npy",id2pro)

40443199
阈值为 0
40443199
1 0.0
1 0.0012363018068872345
1 0.002472603613774469
1 0.003708905420661704
1 0.004945207227548938
1 0.0061815090344361735
1 0.007417810841323408
1 0.008654112648210642
1 0.009890414455097876
1 0.01112671626198511
1 0.012363018068872347
1 0.013599319875759582
1 0.014835621682646816
1 0.01607192348953405
1 0.017308225296421283
1 0.01854452710330852
1 0.019780828910195752
1 0.02101713071708299
1 0.02225343252397022
1 0.023489734330857458
1 0.024726036137744694
1 0.025962337944631927
1 0.027198639751519163
1 0.028434941558406396
1 0.029671243365293632
1 0.030907545172180865
1 0.0321438469790681
1 0.033380148785955334
1 0.03461645059284257
1 0.035852752399729806
1 0.03708905420661704
1 0.03832535601350427
1 0.039561657820391505
1 0.040797959627278745
1 0.04203426143416598
1 0.04327056324105321
1 0.04450686504794044
1 0.04574316685482768
1 0.046979468661714915
1 0.04821577046860215
1 0.04945207227548939
1 0.05068837408237662
1 0.051924675889263854
1 0.053160977696151

1 0.4685583848102619
1 0.4697946866171491
1 0.4710309884240364
1 0.4722672902309236
1 0.47350359203781084
1 0.4747398938446981
1 0.47597619565158533
1 0.47721249745847255
1 0.47844879926535977
1 0.47968510107224704
1 0.48092140287913426
1 0.4821577046860215
1 0.48339400649290876
1 0.484630308299796
1 0.4858666101066832
1 0.4871029119135704
1 0.4883392137204577
1 0.4895755155273449
1 0.4908118173342321
1 0.4920481191411194
1 0.4932844209480066
1 0.49452072275489384
1 0.49575702456178106
1 0.49699332636866833
1 0.49822962817555555
1 0.49946592998244277
1 0.50070223178933
1 0.5019385335962172
1 0.5031748354031045
1 0.5044111372099918
1 0.5056474390168789
1 0.5068837408237662
1 0.5081200426306535
1 0.5093563444375406
1 0.5105926462444279
1 0.5118289480513152
1 0.5130652498582023
1 0.5143015516650896
1 0.5155378534719768
1 0.5167741552788641
1 0.5180104570857513
1 0.5192467588926385
1 0.5204830606995258
1 0.521719362506413
1 0.5229556643133002
1 0.5241919661201875
1 0.5254282679270748
1 0.5

1 0.9544249949169451
1 0.9556612967238324
1 0.9568975985307195
1 0.9581339003376068
1 0.9593702021444941
1 0.9606065039513813
1 0.9618428057582685
1 0.9630791075651558
1 0.964315409372043
1 0.9655517111789302
1 0.9667880129858175
1 0.9680243147927047
1 0.969260616599592
1 0.9704969184064792
1 0.9717332202133664
1 0.9729695220202537
1 0.9742058238271408
1 0.9754421256340281
1 0.9766784274409154
1 0.9779147292478025
1 0.9791510310546898
1 0.9803873328615771
1 0.9816236346684643
1 0.9828599364753515
1 0.9840962382822388
1 0.985332540089126
1 0.9865688418960132
1 0.9878051437029004
1 0.9890414455097877
1 0.990277747316675
1 0.9915140491235621
1 0.9927503509304494
1 0.9939866527373367
1 0.9952229545442238
1 0.9964592563511111
1 0.9976955581579984
1 0.9989318599648855
21778332
阈值为 0
21778332
2 0.0
2 0.0022958599400541785
2 0.004591719880108357
2 0.0068875798201625356
2 0.009183439760216714
2 0.011479299700270893
2 0.013775159640325071
2 0.01607101958037925
2 0.018366879520433428
2 0.02066273

2 0.8012551190789083
2 0.8035509790189626
2 0.8058468389590167
2 0.8081426988990709
2 0.8104385588391251
2 0.8127344187791792
2 0.8150302787192334
2 0.8173261386592876
2 0.8196219985993418
2 0.8219178585393959
2 0.8242137184794501
2 0.8265095784195043
2 0.8288054383595584
2 0.8311012982996127
2 0.8333971582396669
2 0.835693018179721
2 0.8379888781197752
2 0.8402847380598294
2 0.8425805979998836
2 0.8448764579399377
2 0.8471723178799919
2 0.8494681778200461
2 0.8517640377601002
2 0.8540598977001544
2 0.8563557576402087
2 0.8586516175802628
2 0.860947477520317
2 0.8632433374603712
2 0.8655391974004253
2 0.8678350573404795
2 0.8701309172805337
2 0.8724267772205879
2 0.874722637160642
2 0.8770184971006962
2 0.8793143570407504
2 0.8816102169808046
2 0.8839060769208588
2 0.886201936860913
2 0.8884977968009671
2 0.8907936567410213
2 0.8930895166810755
2 0.8953853766211297
2 0.8976812365611838
2 0.899977096501238
2 0.9022729564412922
2 0.9045688163813463
2 0.9068646763214006
2 0.90916053626145

6 0.4237772505071201
6 0.42921029218028833
6 0.43464333385345655
6 0.4400763755266247
6 0.4455094171997929
6 0.45094245887296114
6 0.45637550054612935
6 0.46180854221929757
6 0.4672415838924658
6 0.47267462556563394
6 0.47810766723880216
6 0.4835407089119704
6 0.4889737505851386
6 0.4944067922583068
6 0.499839833931475
6 0.5052728756046432
6 0.5107059172778114
6 0.5161389589509796
6 0.5215720006241479
6 0.527005042297316
6 0.5324380839704842
6 0.5378711256436525
6 0.5433041673168206
6 0.5487372089899889
6 0.5541702506631571
6 0.5596032923363252
6 0.5650363340094935
6 0.5704693756826617
6 0.5759024173558299
6 0.5813354590289981
6 0.5867685007021664
6 0.5922015423753345
6 0.5976345840485027
6 0.6030676257216709
6 0.6085006673948391
6 0.6139337090680074
6 0.6193667507411755
6 0.6247997924143438
6 0.630232834087512
6 0.6356658757606801
6 0.6410989174338484
6 0.6465319591070166
6 0.6519650007801848
6 0.657398042453353
6 0.6628310841265213
6 0.6682641257996894
6 0.6736971674728576
6 0.679130

7 0.8025898116564488
7 0.8054460031570055
7 0.8083021946575624
7 0.8111583861581191
7 0.8140145776586759
7 0.8168707691592326
7 0.8197269606597893
7 0.8225831521603462
7 0.8254393436609029
7 0.8282955351614596
7 0.8311517266620164
7 0.8340079181625731
7 0.83686410966313
7 0.8397203011636867
7 0.8425764926642434
7 0.8454326841648002
7 0.8482888756653569
7 0.8511450671659138
7 0.8540012586664705
7 0.8568574501670272
7 0.859713641667584
7 0.8625698331681407
7 0.8654260246686974
7 0.8682822161692543
7 0.871138407669811
7 0.8739945991703678
7 0.8768507906709245
7 0.8797069821714812
7 0.8825631736720381
7 0.8854193651725948
7 0.8882755566731515
7 0.8911317481737083
7 0.893987939674265
7 0.8968441311748219
7 0.8997003226753786
7 0.9025565141759353
7 0.9054127056764921
7 0.9082688971770488
7 0.9111250886776057
7 0.9139812801781624
7 0.9168374716787191
7 0.9196936631792759
7 0.9225498546798326
7 0.9254060461803894
7 0.9282622376809462
7 0.9311184291815029
7 0.9339746206820597
7 0.93683081218261

9 0.8568640122104206
9 0.8604794721775532
9 0.8640949321446858
9 0.8677103921118183
9 0.871325852078951
9 0.8749413120460835
9 0.8785567720132161
9 0.8821722319803487
9 0.8857876919474813
9 0.8894031519146138
9 0.8930186118817464
9 0.896634071848879
9 0.9002495318160115
9 0.9038649917831442
9 0.9074804517502767
9 0.9110959117174093
9 0.9147113716845419
9 0.9183268316516744
9 0.921942291618807
9 0.9255577515859396
9 0.9291732115530722
9 0.9327886715202047
9 0.9364041314873374
9 0.9400195914544699
9 0.9436350514216024
9 0.9472505113887351
9 0.9508659713558676
9 0.9544814313230002
9 0.9580968912901328
9 0.9617123512572654
9 0.9653278112243979
9 0.9689432711915306
9 0.9725587311586631
9 0.9761741911257956
9 0.9797896510929283
9 0.9834051110600608
9 0.9870205710271934
9 0.990636030994326
9 0.9942514909614586
9 0.9978669509285911
100000
0.0000% 0.9390077590942383


/root/userfolder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


1.0000% 287.6226031780243
2.0000% 368.16080927848816
3.0000% 365.93217945098877
4.0000% 396.9899654388428
5.0000% 425.5178327560425
6.0000% 411.471479177475
7.0000% 386.06273555755615
8.0000% 381.87246656417847
9.0000% 431.15779519081116
10.0000% 323.7781903743744
11.0000% 340.8690593242645
12.0000% 482.0058979988098
13.0000% 443.3359341621399
14.0000% 489.2805254459381
15.0000% 481.41703748703003
16.0000% 360.35302448272705
17.0000% 374.2408308982849
18.0000% 406.17063784599304
19.0000% 487.3118095397949
20.0000% 498.6573212146759
21.0000% 382.0263659954071
22.0000% 361.7519726753235
23.0000% 366.8185224533081
24.0000% 420.5768554210663
25.0000% 455.63420486450195
26.0000% 444.9125225543976
27.0000% 302.3919072151184
28.0000% 318.3100175857544
29.0000% 330.4357602596283
30.0000% 622.7125780582428
31.0000% 580.7988085746765
32.0000% 455.24184918403625
33.0000% 477.7453730106354
34.0000% 404.3985617160797
35.0000% 386.61381316185
36.0000% 402.5511906147003
37.0000% 670.7976543903351
38.

In [11]:
# 验证集代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
T = 0
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_csv("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    table.columns = ['user_id','num']
    #T = table['num'].mean()
    print(len(table))
    print("阈值为 %d"%T)
    print(len(table[table.num >T]))
    table = table[table.num >T]
    temp=table.values[:,:]
    user_num=temp.shape[0]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        user2hours[i-1][temp[j,0]]=int(temp[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

id2label=np.zeros([50000])
id2pro=np.zeros([50000,9])
id2num=np.zeros([50000,9])
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("val")
files.sort()
all_cnt=len(files)
print(all_cnt)
cnt=0
acc_cnt=0
last_time=time()
for file in files:
    s=file.split(".txt")[0].split("/")[-1].split("_")
    reg_id=int(s[0])
    true_label=int(s[1])
    table = pd.read_table(file, header=None)
    table.columns = ["uid","num"]
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        L = table2cnt[i,1].split(',')
        day = 0
        for item in L: 
            d = dt.datetime.strptime(item.split('&')[0],"%Y%m%d")
            if d.weekday() in [5,6]: day+=1
        table2cnt[i,1]=day
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0   #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user):
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label
    id2num[cnt]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[cnt]=pre_label
    id2pro[cnt]=pro[:]
    if true_label==pre_label:
        acc_cnt+=1
    cnt+=1
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),"val acc ",acc_cnt/cnt,now_time-last_time)
        last_time=time()

np.save("id2num_val.npy",id2num)
np.save("id2label_val.npy",id2label)
np.save("id2pro_val.npy",id2pro)

40443199
阈值为 0
40443199
1 0.0
1 0.0012363018068872345
1 0.002472603613774469
1 0.003708905420661704
1 0.004945207227548938
1 0.0061815090344361735
1 0.007417810841323408
1 0.008654112648210642
1 0.009890414455097876
1 0.01112671626198511
1 0.012363018068872347
1 0.013599319875759582
1 0.014835621682646816
1 0.01607192348953405
1 0.017308225296421283
1 0.01854452710330852
1 0.019780828910195752
1 0.02101713071708299
1 0.02225343252397022
1 0.023489734330857458
1 0.024726036137744694
1 0.025962337944631927
1 0.027198639751519163
1 0.028434941558406396
1 0.029671243365293632
1 0.030907545172180865
1 0.0321438469790681
1 0.033380148785955334
1 0.03461645059284257
1 0.035852752399729806
1 0.03708905420661704
1 0.03832535601350427
1 0.039561657820391505
1 0.040797959627278745
1 0.04203426143416598
1 0.04327056324105321
1 0.04450686504794044
1 0.04574316685482768
1 0.046979468661714915
1 0.04821577046860215
1 0.04945207227548939
1 0.05068837408237662
1 0.051924675889263854
1 0.053160977696151

1 0.4685583848102619
1 0.4697946866171491
1 0.4710309884240364
1 0.4722672902309236
1 0.47350359203781084
1 0.4747398938446981
1 0.47597619565158533
1 0.47721249745847255
1 0.47844879926535977
1 0.47968510107224704
1 0.48092140287913426
1 0.4821577046860215
1 0.48339400649290876
1 0.484630308299796
1 0.4858666101066832
1 0.4871029119135704
1 0.4883392137204577
1 0.4895755155273449
1 0.4908118173342321
1 0.4920481191411194
1 0.4932844209480066
1 0.49452072275489384
1 0.49575702456178106
1 0.49699332636866833
1 0.49822962817555555
1 0.49946592998244277
1 0.50070223178933
1 0.5019385335962172
1 0.5031748354031045
1 0.5044111372099918
1 0.5056474390168789
1 0.5068837408237662
1 0.5081200426306535
1 0.5093563444375406
1 0.5105926462444279
1 0.5118289480513152
1 0.5130652498582023
1 0.5143015516650896
1 0.5155378534719768
1 0.5167741552788641
1 0.5180104570857513
1 0.5192467588926385
1 0.5204830606995258
1 0.521719362506413
1 0.5229556643133002
1 0.5241919661201875
1 0.5254282679270748
1 0.5

1 0.9519523913031707
1 0.9531886931100579
1 0.9544249949169451
1 0.9556612967238324
1 0.9568975985307195
1 0.9581339003376068
1 0.9593702021444941
1 0.9606065039513813
1 0.9618428057582685
1 0.9630791075651558
1 0.964315409372043
1 0.9655517111789302
1 0.9667880129858175
1 0.9680243147927047
1 0.969260616599592
1 0.9704969184064792
1 0.9717332202133664
1 0.9729695220202537
1 0.9742058238271408
1 0.9754421256340281
1 0.9766784274409154
1 0.9779147292478025
1 0.9791510310546898
1 0.9803873328615771
1 0.9816236346684643
1 0.9828599364753515
1 0.9840962382822388
1 0.985332540089126
1 0.9865688418960132
1 0.9878051437029004
1 0.9890414455097877
1 0.990277747316675
1 0.9915140491235621
1 0.9927503509304494
1 0.9939866527373367
1 0.9952229545442238
1 0.9964592563511111
1 0.9976955581579984
1 0.9989318599648855
21778332
阈值为 0
21778332
2 0.0
2 0.0022958599400541785
2 0.004591719880108357
2 0.0068875798201625356
2 0.009183439760216714
2 0.011479299700270893
2 0.013775159640325071
2 0.01607101958

2 0.7966633991988
2 0.7989592591388541
2 0.8012551190789083
2 0.8035509790189626
2 0.8058468389590167
2 0.8081426988990709
2 0.8104385588391251
2 0.8127344187791792
2 0.8150302787192334
2 0.8173261386592876
2 0.8196219985993418
2 0.8219178585393959
2 0.8242137184794501
2 0.8265095784195043
2 0.8288054383595584
2 0.8311012982996127
2 0.8333971582396669
2 0.835693018179721
2 0.8379888781197752
2 0.8402847380598294
2 0.8425805979998836
2 0.8448764579399377
2 0.8471723178799919
2 0.8494681778200461
2 0.8517640377601002
2 0.8540598977001544
2 0.8563557576402087
2 0.8586516175802628
2 0.860947477520317
2 0.8632433374603712
2 0.8655391974004253
2 0.8678350573404795
2 0.8701309172805337
2 0.8724267772205879
2 0.874722637160642
2 0.8770184971006962
2 0.8793143570407504
2 0.8816102169808046
2 0.8839060769208588
2 0.886201936860913
2 0.8884977968009671
2 0.8907936567410213
2 0.8930895166810755
2 0.8953853766211297
2 0.8976812365611838
2 0.899977096501238
2 0.9022729564412922
2 0.9045688163813463


6 0.4074781254876155
6 0.4129111671607837
6 0.4183442088339519
6 0.4237772505071201
6 0.42921029218028833
6 0.43464333385345655
6 0.4400763755266247
6 0.4455094171997929
6 0.45094245887296114
6 0.45637550054612935
6 0.46180854221929757
6 0.4672415838924658
6 0.47267462556563394
6 0.47810766723880216
6 0.4835407089119704
6 0.4889737505851386
6 0.4944067922583068
6 0.499839833931475
6 0.5052728756046432
6 0.5107059172778114
6 0.5161389589509796
6 0.5215720006241479
6 0.527005042297316
6 0.5324380839704842
6 0.5378711256436525
6 0.5433041673168206
6 0.5487372089899889
6 0.5541702506631571
6 0.5596032923363252
6 0.5650363340094935
6 0.5704693756826617
6 0.5759024173558299
6 0.5813354590289981
6 0.5867685007021664
6 0.5922015423753345
6 0.5976345840485027
6 0.6030676257216709
6 0.6085006673948391
6 0.6139337090680074
6 0.6193667507411755
6 0.6247997924143438
6 0.630232834087512
6 0.6356658757606801
6 0.6410989174338484
6 0.6465319591070166
6 0.6519650007801848
6 0.657398042453353
6 0.662831

7 0.7997336201558921
7 0.8025898116564488
7 0.8054460031570055
7 0.8083021946575624
7 0.8111583861581191
7 0.8140145776586759
7 0.8168707691592326
7 0.8197269606597893
7 0.8225831521603462
7 0.8254393436609029
7 0.8282955351614596
7 0.8311517266620164
7 0.8340079181625731
7 0.83686410966313
7 0.8397203011636867
7 0.8425764926642434
7 0.8454326841648002
7 0.8482888756653569
7 0.8511450671659138
7 0.8540012586664705
7 0.8568574501670272
7 0.859713641667584
7 0.8625698331681407
7 0.8654260246686974
7 0.8682822161692543
7 0.871138407669811
7 0.8739945991703678
7 0.8768507906709245
7 0.8797069821714812
7 0.8825631736720381
7 0.8854193651725948
7 0.8882755566731515
7 0.8911317481737083
7 0.893987939674265
7 0.8968441311748219
7 0.8997003226753786
7 0.9025565141759353
7 0.9054127056764921
7 0.9082688971770488
7 0.9111250886776057
7 0.9139812801781624
7 0.9168374716787191
7 0.9196936631792759
7 0.9225498546798326
7 0.9254060461803894
7 0.9282622376809462
7 0.9311184291815029
7 0.93397462068205

9 0.8460176323090229
9 0.8496330922761555
9 0.8532485522432881
9 0.8568640122104206
9 0.8604794721775532
9 0.8640949321446858
9 0.8677103921118183
9 0.871325852078951
9 0.8749413120460835
9 0.8785567720132161
9 0.8821722319803487
9 0.8857876919474813
9 0.8894031519146138
9 0.8930186118817464
9 0.896634071848879
9 0.9002495318160115
9 0.9038649917831442
9 0.9074804517502767
9 0.9110959117174093
9 0.9147113716845419
9 0.9183268316516744
9 0.921942291618807
9 0.9255577515859396
9 0.9291732115530722
9 0.9327886715202047
9 0.9364041314873374
9 0.9400195914544699
9 0.9436350514216024
9 0.9472505113887351
9 0.9508659713558676
9 0.9544814313230002
9 0.9580968912901328
9 0.9617123512572654
9 0.9653278112243979
9 0.9689432711915306
9 0.9725587311586631
9 0.9761741911257956
9 0.9797896510929283
9 0.9834051110600608
9 0.9870205710271934
9 0.990636030994326
9 0.9942514909614586
9 0.9978669509285911
50000


/root/userfolder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


2.0000% val acc  0.704 462.27247071266174
4.0000% val acc  0.69 457.7440960407257
6.0000% val acc  0.6906666666666667 329.71782755851746
8.0000% val acc  0.68975 272.602087020874
10.0000% val acc  0.6902 280.6611421108246
12.0000% val acc  0.6886666666666666 2860.933722257614
14.0000% val acc  0.6892857142857143 471.217325925827
16.0000% val acc  0.688125 397.59897351264954
18.0000% val acc  0.6867777777777778 481.7958650588989
20.0000% val acc  0.6837 427.55074644088745
22.0000% val acc  0.6854545454545454 444.00219225883484
24.0000% val acc  0.6845 441.73290848731995
26.0000% val acc  0.6854615384615385 418.82050251960754
28.0000% val acc  0.6857857142857143 439.28289103507996
30.0000% val acc  0.6854666666666667 461.79651618003845
32.0000% val acc  0.68625 463.35434794425964
34.0000% val acc  0.6864117647058824 416.825320482254
36.0000% val acc  0.6854444444444444 430.59750533103943
38.0000% val acc  0.6863684210526316 364.700847864151
40.0000% val acc  0.68615 328.3861746788025
42.